# Creación de tablas para la RDB

In [1]:
import numpy as np
import sqlite3

import pandas as pd
import numpy as np

# Pandas options
pd.set_option('display.max_columns', 500)

In [2]:
import sqlite3
import os
os.remove("censusdb.db")

In [3]:
def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        return conn
    except Error as e:
        print(e)

    return conn

def create_table(conn, create_table_sql):
    """ create a table from the create_table_sql statement
    :param conn: Connection object
    :param create_table_sql: a CREATE TABLE statement
    :return:
    """
    try:
        c = conn.cursor()
        c.execute(create_table_sql)
    except Error as e:
        print(e)

In [4]:
def main():
    database = r"C:\Users\Jaime\Documents\innovati\REST_API\censusdb.db"

    sql_create_marital_stat_tbl  = """ CREATE TABLE IF NOT EXISTS marital_stat_tbl (
                                        id integer PRIMARY KEY,
                                        marital_stat text NOT NULL
                                    ); """
    
    sql_create_race_tbl  = """ CREATE TABLE IF NOT EXISTS race_tbl (
                                id integer PRIMARY KEY,
                                race text NOT NULL
                            ); """

    sql_create_sex_tbl  = """ CREATE TABLE IF NOT EXISTS sex_tbl (
                                id integer PRIMARY KEY,
                                sex text NOT NULL
                            ); """

    sql_create_hisp_origin_tbl  = """ CREATE TABLE IF NOT EXISTS hisp_origin_tbl (
                                        id integer PRIMARY KEY,
                                        hisp_origin text NOT NULL
                                    ); """

    sql_create_education_tbl  = """ CREATE TABLE IF NOT EXISTS education_tbl (
                                      id integer PRIMARY KEY,
                                      education text NOT NULL
                                 ); """
    
    sql_create_person_tbl = """CREATE TABLE IF NOT EXISTS person_tbl (
                                    id_person integer PRIMARY KEY,
                                    age text NOT NULL,
                                    year integer,
                                    marital_stat integer NOT NULL,
                                    race integer NOT NULL,
                                    education integer NOT NULL,
                                    hisp_origin integer NOT NULL,
                                    sex integer NOT NULL,
                                    FOREIGN KEY (marital_stat) REFERENCES marital_stat_tbl (id),
                                    FOREIGN KEY (race) REFERENCES race_tbl (id),
                                    FOREIGN KEY (education) REFERENCES education_tbl (id),
                                    FOREIGN KEY (hisp_origin) REFERENCES hisp_origin_tbl (id),
                                    FOREIGN KEY (sex) REFERENCES sex_tbl (id)
                                );"""
    
    sql_create_class_worker_tbl  = """ CREATE TABLE IF NOT EXISTS class_worker_tbl (
                                          id integer PRIMARY KEY,
                                          class_worker text NOT NULL
                                     ); """    

    sql_create_major_occ_code_tbl  = """ CREATE TABLE IF NOT EXISTS major_occ_code_tbl (
                                          id integer PRIMARY KEY,
                                          major_occ_code text NOT NULL,
                                          det_ind_code integer,
                                          FOREIGN KEY (det_ind_code) REFERENCES major_ind_code_tbl (det_ind_code)
                                      ); """ 

    sql_create_major_ind_code_tbl  = """ CREATE TABLE IF NOT EXISTS major_ind_code_tbl (
                                          det_ind_code integer PRIMARY KEY,
                                          major_ind_code text NOT NULL
                                      ); """ 
    
    sql_create_employee_tbl = """CREATE TABLE IF NOT EXISTS employee_tbl (
                                    id_person integer NOT NULL,
                                    class_worker integer NOT NULL,
                                    year integer,
                                    det_occ_code integer NOT NULL,
                                    wage_per_hour real,
                                    union_member boolean,
                                    unemp_reason text,
                                    full_or_part_emp text,
                                    own_or_self text,
                                    weeks_worked integer,
                                    income_50k boolean,
                                    FOREIGN KEY (id_person) REFERENCES person_tbl (id_person),
                                    FOREIGN KEY (class_worker) REFERENCES class_worker_tbl (id),
                                    FOREIGN KEY (det_occ_code) REFERENCES major_occ_code_tbl (id)
                                );"""    
    
    # create a database connection
    conn = create_connection(database)

    # create tables
    if conn is not None:
        create_table(conn, sql_create_marital_stat_tbl)
        create_table(conn, sql_create_race_tbl)
        create_table(conn, sql_create_sex_tbl)
        create_table(conn, sql_create_hisp_origin_tbl)
        create_table(conn, sql_create_education_tbl)
        create_table(conn, sql_create_person_tbl)
        create_table(conn, sql_create_class_worker_tbl)
        create_table(conn, sql_create_major_occ_code_tbl)
        create_table(conn, sql_create_major_ind_code_tbl)
        create_table(conn, sql_create_employee_tbl)
    else:
        print("Error! cannot create the database connection.")

In [ ]:
if __name__ == '__main__':
    main()

## Cargamos contenidos y ajustamos las tablas

In [ ]:
census = pd.read_csv("data/census_tidy.csv")
print(census.shape)
census.head()

In [ ]:
census.rename({'Unnamed: 0':'id_person'}, axis=1, inplace=True)
census.drop(['instance_weight'], axis=1)
census.head()

In [ ]:
census['income_50k'].unique().tolist()

In [ ]:
census['income_50k'] = census['income_50k'].replace([' - 50000.', ' 50000+.'],[False,True])
census.head()

In [ ]:
features = pd.read_csv("data/names.csv", sep=';')
features.head()

In [ ]:
def whitespace_remover(dataframe):
    
    # iterating over the columns
    for i in text_fields:
          
        # checking datatype of each columns
        if dataframe[i].dtype == 'object':
              
            # applying strip function on column
            dataframe[i] = dataframe[i].map(str.lstrip)
        else:
              
            # if condn. is False then it will do nothing.
            pass
    return dataframe

In [ ]:
text_fields = features[features.DataType=='text'].Code.tolist()
census = whitespace_remover(census)
census.head()

## Tablas secundarias

Creemos las tablas secundarias

In [ ]:
conn = sqlite3.connect('censusdb.db')  
c = conn.cursor()

In [ ]:
sex_categ = census.sex.unique().tolist()
sex_code = list(range(len(sex_categ)))
sex_tbl = pd.DataFrame(list(zip(sex_code, sex_categ)),
                       columns =['id', 'sex'])
sex_tbl.to_csv('data/tables/sex_tbl.csv', index=False)
sex_tbl.to_sql('sex_tbl', conn, if_exists='append', index = False)
sex_tbl.head()

In [ ]:
hisp_origin_categ = census.hisp_origin.unique().tolist()
hisp_origin_code = list(range(len(hisp_origin_categ)))
hisp_origin_tbl = pd.DataFrame(list(zip(hisp_origin_code, hisp_origin_categ)),
                       columns =['id', 'hisp_origin'])
hisp_origin_tbl.to_csv('data/tables/hisp_origin_tbl.csv', index=False)
hisp_origin_tbl.to_sql('hisp_origin_tbl', conn, if_exists='append', index = False)
hisp_origin_tbl.head()

In [ ]:
martial_status_categ = census.marital_stat.unique().tolist()
martial_status_code = list(range(len(martial_status_categ)))
martial_status_tbl = pd.DataFrame(list(zip(martial_status_code, martial_status_categ)),
                       columns =['id', 'marital_stat'])
martial_status_tbl.to_csv('data/tables/martial_status_tbl.csv', index=False)
martial_status_tbl.to_sql('martial_status_tbl', conn, if_exists='append', index = False)
martial_status_tbl.head()

In [ ]:
race_categ = census.race.unique().tolist()
race_code = list(range(len(race_categ)))
race_tbl = pd.DataFrame(list(zip(race_code, race_categ)),
                       columns =['id', 'race'])
race_tbl.to_csv('data/tables/race_tbl.csv', index=False)
race_tbl.to_sql('race_tbl', conn, if_exists='append', index = False)
race_tbl.head()

In [ ]:
class_worker_categ = census.class_worker.unique().tolist()
class_worker_code = list(range(len(class_worker_categ)))
class_worker_tbl = pd.DataFrame(list(zip(class_worker_code, class_worker_categ)),
                               columns =['id', 'class_worker'])
class_worker_tbl.to_csv('data/tables/class_worker_tbl.csv', index=False)
class_worker_tbl.to_sql('class_worker_tbl', conn, if_exists='append', index = False)
class_worker_tbl.head()

In [ ]:
education_categ = census.education.unique().tolist()
education_code = list(range(len(education_categ)))
education_tbl = pd.DataFrame(list(zip(education_code, education_categ)),
                               columns =['id', 'education'])
education_tbl.to_sql('education_tbl', conn, if_exists='append', index = False)
education_tbl.head()

Las tablas de ocupación e industria requieren extraer las categorías y los códigos directamente de la tabla tidy

In [ ]:
det_ind_code = census.det_ind_code.unique().tolist()
det_ind_code_tbl = census[['det_ind_code', 'major_ind_code']].drop_duplicates()
print(det_ind_code_tbl.shape)
det_ind_code_tbl.to_csv('data/tables/det_ind_code_tbl.csv', index=False)
det_ind_code_tbl.to_sql('det_ind_code_tbl', conn, if_exists='append', index = False)
det_ind_code_tbl.head()

In [ ]:
census[census.det_ind_code==1][['det_ind_code', 'major_ind_code', 'major_occ_code']].major_occ_code.unique()

In [ ]:
det_occ_code = census.det_ind_code.unique().tolist()
det_occ_code_tbl = census[['det_occ_code', 'major_occ_code',
                           'det_ind_code']].drop_duplicates(subset=['det_occ_code'])
print(det_occ_code_tbl.shape)
det_occ_code_tbl.to_csv('data/tables/det_ind_code_tbl.csv', index=False)
det_occ_code_tbl.to_sql('det_occ_code_tbl', conn, if_exists='append', index = False)
det_occ_code_tbl.head()

### Tabla `employee_tbl`

In [ ]:
employee_vars = features[features.Employee==1].Code.tolist() + ['id_person']
employee = census[employee_vars]
employee.head()

In [ ]:
employee = employee.merge(class_worker_tbl, on='class_worker', 
                          how='inner')
employee.drop(['class_worker'], axis=1, inplace=True)
employee.rename({'id':'class_worker'}, axis=1, inplace=True)
employee.head()

In [ ]:
# employee['id'] = pd.Series(list(range(len(employee))))
employee.to_csv('data/tables/employee.csv', index=False)
employee.head()

In [ ]:
employee.to_sql('employee_tbl', conn, if_exists='append', index = False)

### Tabla `person`

In [ ]:
person_vars = features[features.Person==1].Code.tolist() + ['id_person']
person = census[person_vars]
person.head()

In [ ]:
person = person.merge(education_tbl, on='education', 
                      how='inner')
person.drop(['education'], axis=1, inplace=True)
person.rename({'id':'education'}, axis=1, inplace=True)
person

In [ ]:
person = person.merge(martial_status_tbl, on='marital_stat', 
                      how='inner')
person.drop(['marital_stat'], axis=1, inplace=True)
person.rename({'id':'marital_stat'}, axis=1, inplace=True)
person

In [ ]:
person = person.merge(race_tbl, on='race', 
                      how='inner')
person.drop(['race'], axis=1, inplace=True)
person.rename({'id':'race'}, axis=1, inplace=True)
person

In [ ]:
person = person.merge(hisp_origin_tbl, on='hisp_origin', 
                      how='inner')
person.drop(['hisp_origin'], axis=1, inplace=True)
person.rename({'id':'hisp_origin'}, axis=1, inplace=True)
person

In [ ]:
person = person.merge(sex_tbl, on='sex', 
                      how='inner')
person.drop(['sex'], axis=1, inplace=True)
person.rename({'id':'sex'}, axis=1, inplace=True)
person

In [ ]:
person.to_csv('data/tables/person.csv', index=False)
person.to_sql('person_tbl', conn, if_exists='append', index = False)

## Queries de prueba

In [ ]:
sex_tbl_query = """
SELECT * FROM sex_tbl;
"""
data = pd.read_sql_query(sex_tbl_query, conn)
data

In [ ]:
age_tbl_query = """
SELECT * FROM person_tbl
WHERE age = 73;
"""
data = pd.read_sql_query(age_tbl_query, conn)
data.head()

In [ ]:
employee_tbl_query = """
SELECT * FROM employee_tbl
WHERE det_occ_code = 45;
"""
data = pd.read_sql_query(employee_tbl_query, conn)
data.head()

In [ ]:
mix_query = """
WITH total_tbl1 AS (SELECT * FROM person_tbl as p
INNER JOIN employee_tbl as e ON e.id_person=p.id_person)
SELECT * FROM hisp_origin_tbl as hsp 
INNER JOIN total_tbl1 ON total_tbl1.hisp_origin = hsp.id
"""
data = pd.read_sql_query(mix_query, conn)
data.head()

In [ ]:
codes_query = """
WITH person_hisp AS (
    SELECT p.id_person, p.age, p.year, p.marital_stat, p.race, p.education, 
           p.sex, hsp.hisp_origin FROM person_tbl as p
    INNER JOIN hisp_origin_tbl as hsp ON hsp.id = p.hisp_origin
)
SELECT r.race, p2.id_person, p2.age, p2.year, p2.marital_stat,
       p2.education, p2.hisp_origin, p2.sex
       FROM race_tbl as r 
INNER JOIN person_hisp as p2 ON p2.race = r.id;
"""
data = pd.read_sql_query(codes_query, conn)
data.head()

In [ ]:
post_query = """
WITH data AS (
    WITH data_occ AS (
        WITH data_class AS(
            WITH person_total AS (
                WITH person_edu AS (
                    WITH person_sex AS (
                        WITH person_race AS (
                            WITH person_hisp AS (
                                SELECT p1.id_person, p1.age, p1.year, p1.marital_stat, p1.race, 
                                p1.education, p1.sex, hsp.hisp_origin FROM person_tbl as p1
                                INNER JOIN hisp_origin_tbl as hsp ON hsp.id = p1.hisp_origin
                            )
                            SELECT r.race, p2.id_person, p2.age, p2.year, p2.marital_stat,
                                    p2.education, p2.hisp_origin, p2.sex FROM race_tbl as r 
                            INNER JOIN person_hisp as p2 ON p2.race = r.id
                            )
                        SELECT p3.id_person, p3.race, p3.age, p3.year, p3.education, p3.hisp_origin,
                                p3.sex, ms.marital_stat FROM person_race AS p3
                        INNER JOIN  martial_status_tbl as ms ON ms.id = p3.marital_stat
                        )
                    SELECT p4.id_person, p4.race, p4.age, p4.year, p4.marital_stat, p4.education, 
                            p4.hisp_origin, sex_tbl.sex FROM person_sex AS p4
                    INNER JOIN sex_tbl ON sex_tbl.id = p4.sex
                )
                SELECT p5.id_person, p5.race, p5.age, p5.year, p5.marital_stat, edu.education,
                        p5.hisp_origin, p5.sex FROM person_edu as p5
                INNER JOIN education_tbl as edu ON edu.id = p5.education
            )
            SELECT p.id_person, p.race, p.age, p.year, p.marital_stat, p.education, p.hisp_origin, 
                    p.sex, e.det_occ_code, e.wage_per_hour, e.union_member, e.unemp_reason,
                    e.own_or_self, e.weeks_worked, e.income_50k, e.class_worker FROM person_total AS p
            INNER JOIN employee_tbl as e ON e.id_person=p.id_person
        )
        SELECT dcl.id_person, dcl.race, dcl.age, dcl.year, dcl.marital_stat, dcl.education, dcl.hisp_origin,
               dcl.sex, dcl.wage_per_hour, dcl.union_member, dcl.unemp_reason, dcl.own_or_self,
               dcl.weeks_worked, dcl.income_50k, dcl.det_occ_code, cw.class_worker FROM data_class as dcl
        INNER JOIN class_worker_tbl as cw ON cw.id = dcl.class_worker
    )
    SELECT docc.id_person, docc.race, docc.age, docc.year, docc.marital_stat, docc.education, docc.hisp_origin,
           docc.sex, docc.wage_per_hour, docc.union_member, docc.unemp_reason, docc.own_or_self,
           docc.weeks_worked, docc.income_50k, mo.major_occ_code, mo.det_ind_code, docc.class_worker
           FROM data_occ as docc
    INNER JOIN det_occ_code_tbl as mo ON mo.det_occ_code = docc.det_occ_code
)
SELECT data.id_person, data.race, data.age, data.year, data.marital_stat, data.education, data.hisp_origin,
       data.sex, data.wage_per_hour, data.union_member, data.unemp_reason, data.own_or_self, data.class_worker,
       data.weeks_worked, data.income_50k, data.major_occ_code, mi.major_ind_code FROM data
INNER JOIN det_ind_code_tbl as mi ON mi.det_ind_code = data.det_ind_code;
"""
data = pd.read_sql_query(post_query, conn)
data.head()

In [ ]:
data.class_worker.unique().tolist()

In [ ]:
data.major_ind_code.unique().tolist()

In [ ]:
data.major_occ_code.unique().tolist()

In [ ]:
data.sex.unique().tolist()

In [ ]:
data.hisp_origin.unique().tolist()

In [ ]:
data.marital_stat.unique().tolist()

In [ ]:
filter_query = """
WITH data AS (
    WITH data_occ AS (
        WITH data_class AS(
            WITH person_total AS (
                WITH person_edu AS (
                    WITH person_sex AS (
                        WITH person_race AS (
                            WITH person_hisp AS (
                                SELECT p1.id_person, p1.age, p1.year, p1.marital_stat, p1.race, 
                                p1.education, p1.sex, hsp.hisp_origin FROM person_tbl as p1
                                INNER JOIN hisp_origin_tbl as hsp ON hsp.id = p1.hisp_origin
                            )
                            SELECT r.race, p2.id_person, p2.age, p2.year, p2.marital_stat,
                                    p2.education, p2.hisp_origin, p2.sex FROM race_tbl as r 
                            INNER JOIN person_hisp as p2 ON p2.race = r.id
                            )
                        SELECT p3.id_person, p3.race, p3.age, p3.year, p3.education, p3.hisp_origin,
                                p3.sex, ms.marital_stat FROM person_race AS p3
                        INNER JOIN  martial_status_tbl as ms ON ms.id = p3.marital_stat
                        )
                    SELECT p4.id_person, p4.race, p4.age, p4.year, p4.marital_stat, p4.education, 
                            p4.hisp_origin, sex_tbl.sex FROM person_sex AS p4
                    INNER JOIN sex_tbl ON sex_tbl.id = p4.sex
                )
                SELECT p5.id_person, p5.race, p5.age, p5.year, p5.marital_stat, edu.education,
                        p5.hisp_origin, p5.sex FROM person_edu as p5
                INNER JOIN education_tbl as edu ON edu.id = p5.education
            )
            SELECT p.id_person, p.race, p.age, p.year, p.marital_stat, p.education, p.hisp_origin, 
                    p.sex, e.det_occ_code, e.wage_per_hour, e.union_member, e.unemp_reason,
                    e.own_or_self, e.weeks_worked, e.income_50k, e.class_worker FROM person_total AS p
            INNER JOIN employee_tbl as e ON e.id_person=p.id_person
        )
        SELECT dcl.id_person, dcl.race, dcl.age, dcl.year, dcl.marital_stat, dcl.education, dcl.hisp_origin,
               dcl.sex, dcl.wage_per_hour, dcl.union_member, dcl.unemp_reason, dcl.own_or_self,
               dcl.weeks_worked, dcl.income_50k, dcl.det_occ_code, cw.class_worker FROM data_class as dcl
        INNER JOIN class_worker_tbl as cw ON cw.id = dcl.class_worker
    )
    SELECT docc.id_person, docc.race, docc.age, docc.year, docc.marital_stat, docc.education, docc.hisp_origin,
           docc.sex, docc.wage_per_hour, docc.union_member, docc.unemp_reason, docc.own_or_self,
           docc.weeks_worked, docc.income_50k, mo.major_occ_code, mo.det_ind_code, docc.class_worker,
           docc.det_occ_code FROM data_occ as docc
    INNER JOIN det_occ_code_tbl as mo ON mo.det_occ_code = docc.det_occ_code
)
SELECT data.id_person, data.race, data.age, data.year, data.marital_stat, data.education, data.hisp_origin,
       data.sex, data.wage_per_hour, data.union_member, data.unemp_reason, data.own_or_self, data.class_worker,
       data.weeks_worked, data.income_50k, data.major_occ_code, mi.major_ind_code, 
       data.det_ind_code, data.det_occ_code FROM data
INNER JOIN det_ind_code_tbl as mi ON mi.det_ind_code = data.det_ind_code
WHERE age = 31 AND class_worker = ' Private' AND major_ind_code = ' Education' AND sex = ' Female'
      AND major_occ_code = ' Professional specialty' AND hisp_origin = ' Mexican (Mexicano)';
"""
data = pd.read_sql_query(filter_query, conn)
data

In [ ]:
filter_query = """
WITH filter AS (
    WITH data AS (
        WITH data_occ AS (
            WITH data_class AS(
                WITH person_total AS (
                    WITH person_edu AS (
                        WITH person_sex AS (
                            WITH person_race AS (
                                WITH person_hisp AS (
                                    SELECT p1.id_person, p1.age, p1.year, p1.marital_stat, p1.race, 
                                    p1.education, p1.sex, hsp.hisp_origin FROM person_tbl as p1
                                    INNER JOIN hisp_origin_tbl as hsp ON hsp.id = p1.hisp_origin
                                )
                                SELECT r.race, p2.id_person, p2.age, p2.year, p2.marital_stat,
                                        p2.education, p2.hisp_origin, p2.sex FROM race_tbl as r 
                                INNER JOIN person_hisp as p2 ON p2.race = r.id
                                )
                            SELECT p3.id_person, p3.race, p3.age, p3.year, p3.education, p3.hisp_origin,
                                    p3.sex, ms.marital_stat FROM person_race AS p3
                            INNER JOIN  martial_status_tbl as ms ON ms.id = p3.marital_stat
                            )
                        SELECT p4.id_person, p4.race, p4.age, p4.year, p4.marital_stat, p4.education, 
                                p4.hisp_origin, sex_tbl.sex FROM person_sex AS p4
                        INNER JOIN sex_tbl ON sex_tbl.id = p4.sex
                    )
                    SELECT p5.id_person, p5.race, p5.age, p5.year, p5.marital_stat, edu.education,
                            p5.hisp_origin, p5.sex FROM person_edu as p5
                    INNER JOIN education_tbl as edu ON edu.id = p5.education
                )
                SELECT p.id_person, p.race, p.age, p.year, p.marital_stat, p.education, p.hisp_origin, 
                        p.sex, e.det_occ_code, e.wage_per_hour, e.union_member, e.unemp_reason,
                        e.own_or_self, e.weeks_worked, e.income_50k, e.class_worker FROM person_total AS p
                INNER JOIN employee_tbl as e ON e.id_person=p.id_person
            )
            SELECT dcl.id_person, dcl.race, dcl.age, dcl.year, dcl.marital_stat, dcl.education, dcl.hisp_origin,
                   dcl.sex, dcl.wage_per_hour, dcl.union_member, dcl.unemp_reason, dcl.own_or_self,
                   dcl.weeks_worked, dcl.income_50k, dcl.det_occ_code, cw.class_worker FROM data_class as dcl
            INNER JOIN class_worker_tbl as cw ON cw.id = dcl.class_worker
        )
        SELECT docc.id_person, docc.race, docc.age, docc.year, docc.marital_stat, docc.education, docc.hisp_origin,
               docc.sex, docc.wage_per_hour, docc.union_member, docc.unemp_reason, docc.own_or_self,
               docc.weeks_worked, docc.income_50k, mo.major_occ_code, mo.det_ind_code, docc.class_worker,
               docc.det_occ_code FROM data_occ as docc
        INNER JOIN det_occ_code_tbl as mo ON mo.det_occ_code = docc.det_occ_code
    )
    SELECT data.id_person, data.race, data.age, data.year, data.marital_stat, data.education, data.hisp_origin,
       data.sex, data.wage_per_hour, data.union_member, data.unemp_reason, data.own_or_self, data.class_worker,
       data.weeks_worked, data.income_50k, data.major_occ_code, mi.major_ind_code, 
       data.det_ind_code, data.det_occ_code FROM data
    INNER JOIN det_ind_code_tbl as mi ON mi.det_ind_code = data.det_ind_code
    WHERE age = 31 AND class_worker = ' Private' AND major_ind_code = ' Education' AND sex = ' Female'
          AND major_occ_code = ' Professional specialty' AND hisp_origin = ' Mexican (Mexicano)'
)
SELECT avg(wage_per_hour) as mean_wage, avg(weeks_worked) as mean_weeks_worked, 
       min(wage_per_hour) as min_wage, min(weeks_worked) as min_weeks_worked, 
       max(wage_per_hour) as max_wage, max(weeks_worked) as max_weeks_worked, 
       sum(income_50k) as person_50k_plus, count(id_person) as num_person      
FROM filter;
"""
data = pd.read_sql_query(filter_query, conn)
data

In [ ]:
filter_query = """
WITH filter AS (
    WITH data AS (
        WITH data_occ AS (
            WITH data_class AS(
                WITH person_total AS (
                    WITH person_edu AS (
                        WITH person_sex AS (
                            WITH person_race AS (
                                WITH person_hisp AS (
                                    SELECT p1.id_person, p1.age, p1.year, p1.marital_stat, p1.race, 
                                    p1.education, p1.sex, hsp.hisp_origin FROM person_tbl as p1
                                    INNER JOIN hisp_origin_tbl as hsp ON hsp.id = p1.hisp_origin
                                )
                                SELECT r.race, p2.id_person, p2.age, p2.year, p2.marital_stat,
                                        p2.education, p2.hisp_origin, p2.sex FROM race_tbl as r 
                                INNER JOIN person_hisp as p2 ON p2.race = r.id
                                )
                            SELECT p3.id_person, p3.race, p3.age, p3.year, p3.education, p3.hisp_origin,
                                    p3.sex, ms.marital_stat FROM person_race AS p3
                            INNER JOIN  martial_status_tbl as ms ON ms.id = p3.marital_stat
                            )
                        SELECT p4.id_person, p4.race, p4.age, p4.year, p4.marital_stat, p4.education, 
                                p4.hisp_origin, sex_tbl.sex FROM person_sex AS p4
                        INNER JOIN sex_tbl ON sex_tbl.id = p4.sex
                    )
                    SELECT p5.id_person, p5.race, p5.age, p5.year, p5.marital_stat, edu.education,
                            p5.hisp_origin, p5.sex FROM person_edu as p5
                    INNER JOIN education_tbl as edu ON edu.id = p5.education
                )
                SELECT p.id_person, p.race, p.age, p.year, p.marital_stat, p.education, p.hisp_origin, 
                        p.sex, e.det_occ_code, e.wage_per_hour, e.union_member, e.unemp_reason,
                        e.own_or_self, e.weeks_worked, e.income_50k, e.class_worker FROM person_total AS p
                INNER JOIN employee_tbl as e ON e.id_person=p.id_person
            )
            SELECT dcl.id_person, dcl.race, dcl.age, dcl.year, dcl.marital_stat, dcl.education, dcl.hisp_origin,
                   dcl.sex, dcl.wage_per_hour, dcl.union_member, dcl.unemp_reason, dcl.own_or_self,
                   dcl.weeks_worked, dcl.income_50k, dcl.det_occ_code, cw.class_worker FROM data_class as dcl
            INNER JOIN class_worker_tbl as cw ON cw.id = dcl.class_worker
        )
        SELECT docc.id_person, docc.race, docc.age, docc.year, docc.marital_stat, docc.education, docc.hisp_origin,
               docc.sex, docc.wage_per_hour, docc.union_member, docc.unemp_reason, docc.own_or_self,
               docc.weeks_worked, docc.income_50k, mo.major_occ_code, mo.det_ind_code, docc.class_worker,
               docc.det_occ_code FROM data_occ as docc
        INNER JOIN det_occ_code_tbl as mo ON mo.det_occ_code = docc.det_occ_code
    )
    SELECT data.id_person, data.race, data.age, data.year, data.marital_stat, data.education, data.hisp_origin,
       data.sex, data.wage_per_hour, data.union_member, data.unemp_reason, data.own_or_self, data.class_worker,
       data.weeks_worked, data.income_50k, data.major_occ_code, mi.major_ind_code, 
       data.det_ind_code, data.det_occ_code FROM data
    INNER JOIN det_ind_code_tbl as mi ON mi.det_ind_code = data.det_ind_code
    WHERE age = 31 AND class_worker = ' Private' AND data.det_ind_code = 43 AND data.det_occ_code = 7 AND 
          marital_stat = ' Never married' AND
          major_ind_code = ' Education' AND major_occ_code = ' Professional specialty' AND 
          hisp_origin = ' Mexican (Mexicano)' AND sex = ' Female'
)
SELECT avg(wage_per_hour) as mean_wage, avg(weeks_worked) as mean_weeks_worked, 
       min(wage_per_hour) as min_wage, min(weeks_worked) as min_weeks_worked, 
       max(wage_per_hour) as max_wage, max(weeks_worked) as max_weeks_worked, 
       sum(income_50k) as person_50k_plus, count(id_person) as num_person      
FROM filter;
"""
data = pd.read_sql_query(filter_query, conn)
data

In [ ]:
conn.close()